In [1]:
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
!wget http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip
!unzip -q DIV2K_train_HR.zip
!unzip -q DIV2K_train_LR_bicubic_X4.zip

--2020-02-14 04:28:17--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip [following]
--2020-02-14 04:28:18--  https://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3530603713 (3.3G) [application/zip]
Saving to: ‘DIV2K_train_HR.zip’

DIV2K_train_HR.zip  100%[===================>]   3.29G  18.2MB/s    in 3m 6s   

2020-02-14 04:31:25 (18.1 MB/s) - ‘DIV2K_train_HR.zip’ saved [3530603713/3530603713]

--2020-02-14 04:31:26--  http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X4.zip
Resolving data.vision.ee.ethz.ch (data.vision.

In [0]:
!unzip -q src.zip

In [1]:
%tensorflow_version 2.x
import os, random
import math, functools
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from src.dataset import *
from src.models import *
from matplotlib import pyplot as plt
AUTOTUNE = tf.data.experimental.AUTOTUNE

TensorFlow 2.x selected.


In [2]:
tfrecord_creator = TFRecordCreator('./DIV2K_train_HR', './DIV2K_train_LR_bicubic/X4')
tfrecord_creator.make_tfrecord_file('div2d_4x_bicubic.tfrecord')

100%|██████████| 800/800 [01:48<00:00,  7.37it/s]


In [3]:
sr_dataset = SRTfrecordDataset(512)
dataset = sr_dataset.get_dataset('div2d_4x_bicubic.tfrecord', 8, 1024)
dataset

<PrefetchDataset shapes: ((8, 128, 128, 3), (8, 512, 512, 3)), types: (tf.float32, tf.float32)>

In [4]:
sample_x, sample_y = next(iter(dataset))
sample_x = sample_x.numpy()
sample_y = sample_y.numpy()
print('Sample X shape: {}'.format(sample_x.shape))
print('Sample Y shape: {}'.format(sample_y.shape))

Sample X shape: (8, 128, 128, 3)
Sample Y shape: (8, 512, 512, 3)


In [5]:
sample_generator = Generator(128, 3)
sample_sr = sample_generator(sample_x)
print('Sample Generated Image Shape: {}'.format(sample_sr.shape))

Sample Generated Image Shape: (8, 512, 512, 3)


In [7]:
sample_discriminator = Discriminator(512, 3)
sample_discriminator_output = sample_discriminator(sample_sr)
print('Sample Discriminator Output: {}'.format(sample_discriminator_output.shape))

Sample Discriminator Output: (8, 1)
